In [1]:
import pandas as pd 
import numpy as np
from tqdm import trange
from WindPy import w
import datetime

In [2]:
# 获取期货交割日
strikedate = pd.read_excel('data/沪深300分红.xlsx', sheet_name='交割日')['交割日期']

In [3]:
# 合并每年的分红数据
div = []
for x in [2019,2020,2021,2022]:
    div.append(pd.read_excel('data/沪深300分红.xlsx', sheet_name=f'{x}分红预案'))
divs = pd.concat(div, axis=0, ignore_index=True)
divs['除权除息日'] = divs['除权除息日'].map(lambda x: pd.to_datetime(x))
divs = divs.sort_values(by='除权除息日')
divs.reset_index(drop=True, inplace=True)
# divs['分红'] = divs['现金分红'].map(lambda x: 0 if x.startswith('-') else x)
divs['分红'] = divs['现金分红'].map(lambda x: float(x[:-3]) if (isinstance(x, str)and not x.startswith('-')) else 0)
# print(divs)
#     print(div)

             代码    简称      除权除息日     现金分红    分红
0     002607.SZ  中公教育 2019-01-08  0.39元/股  0.39
1     000651.SZ  格力电器 2019-02-25  0.60元/股  0.60
2     600276.SH  恒瑞医药 2019-03-28  0.22元/股  0.22
3     300033.SZ   同花顺 2019-03-29  0.48元/股  0.48
4     600887.SH  伊利股份 2019-04-08  0.70元/股  0.70
...         ...   ...        ...      ...   ...
1015  300274.SZ  阳光电源 2022-06-28  0.11元/股  0.11
1016  600111.SH  北方稀土 2022-06-28  0.45元/股  0.45
1017  600010.SH  包钢股份 2022-06-29  0.02元/股  0.02
1018  601818.SH  光大银行 2022-06-29  0.20元/股  0.20
1019  601628.SH  中国人寿 2022-07-14  0.65元/股  0.65

[1020 rows x 5 columns]


In [4]:
# 确定每笔分红对应的期货交割日
divs['对应交割日'] = divs['除权除息日']
row = 1
totalrow = len(divs)
if divs.loc[0, '除权除息日'] <= strikedate[0]:
    divs.loc[0, '对应交割日'] = strikedate[0]
for x in trange(len(strikedate)-1):
    start = strikedate[x]
    end = strikedate[x+1]
    for x in range(row, totalrow):
        s = divs.loc[x]
        if s['除权除息日'] > start and s['除权除息日'] <= end:
            divs.loc[x, '对应交割日'] = end

        elif s['除权除息日'] > end:
            row = max(x-1,0)
            break
print(divs)

100%|█████████████████████████████████████████████████████████████████████████████████| 47/47 [00:00<00:00, 193.37it/s]

             代码    简称      除权除息日     现金分红    分红      对应交割日
0     002607.SZ  中公教育 2019-01-08  0.39元/股  0.39 2019-01-18
1     000651.SZ  格力电器 2019-02-25  0.60元/股  0.60 2019-03-15
2     600276.SH  恒瑞医药 2019-03-28  0.22元/股  0.22 2019-04-19
3     300033.SZ   同花顺 2019-03-29  0.48元/股  0.48 2019-04-19
4     600887.SH  伊利股份 2019-04-08  0.70元/股  0.70 2019-04-19
...         ...   ...        ...      ...   ...        ...
1015  300274.SZ  阳光电源 2022-06-28  0.11元/股  0.11 2022-07-15
1016  600111.SH  北方稀土 2022-06-28  0.45元/股  0.45 2022-07-15
1017  600010.SH  包钢股份 2022-06-29  0.02元/股  0.02 2022-07-15
1018  601818.SH  光大银行 2022-06-29  0.20元/股  0.20 2022-07-15
1019  601628.SH  中国人寿 2022-07-14  0.65元/股  0.65 2022-07-15

[1020 rows x 6 columns]


In [5]:
# divs.to_excel('分红对应交割日.xlsx')

In [6]:
weight = {} # 导入不同时间段的权重数据
for i in [2019, 2020, 2021,2022]:
    for j in [1,2]:
#         if (i,j) == (2022,2):
#             break
        weight[(i,j)] = pd.read_excel(f'data/hs{i}{j}.xlsx')[['代码', '权重']]
        weight[(i,j)]['权重'] = weight[(i,j)]['权重']/100
# print(weight[(2019,1)])
# 定义权重选择函数
def chooseweight(date, weight=weight):
    datestr = date.strftime('%Y%m%d')
    year = int(datestr[:4])
    mon = int(datestr[4:6])
    if mon <= 6:
        half = 1
    else:
        half = 2
    return weight[(year, half)]

In [7]:
nowtime = datetime.datetime.today()
for x in range(len(strikedate)-1):
    start = strikedate[x]
    end = strikedate[x+1]
    if nowtime > start and nowtime <= end:
        yearmonth = end
        break
numofmonth = (yearmonth.year - 2019)*12 + yearmonth.month
print(numofmonth)

43


In [8]:
# 获取每只股票在上一个月的收盘价， 拼接上权重
w.start()
table = []
## 这里需要注意月数numfmonth

for x in trange(2, numofmonth):
    stockpricedate = strikedate[x-1]
    maturedate0 = strikedate[x]
    maturedate = maturedate0.strftime('%Y%m%d')
    stocks = divs[divs['对应交割日'] == maturedate]
    if len(stocks) == 0:
        continue
    weight = chooseweight(maturedate0)
    stocks.drop_duplicates(subset=['代码'], inplace=True)
    stocks.reset_index(drop=True, inplace=True)
    codes = ','.join(stocks['代码'])
    data = w.wss(codes, "close",f"tradeDate={maturedate};priceAdj=U;cycle=D")
#     print(data)
    temp = pd.DataFrame(data.Data).T
    temp.columns = ['收盘价']
#     temp.index = stocks['代码'] 
    stocks['上期收盘价'] = temp['收盘价']
    stocks = stocks.merge(weight, on='代码', how='left')
    table.append(stocks)
result1 = pd.concat(table, ignore_index=True)
w.close()
# print(result1)


Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


  0%|                                                                                           | 0/41 [00:00<?, ?it/s]D:\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\25286\AppData\Local\Temp/ipykernel_11220/2023034410.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks['上期收盘价'] = temp['收盘价']
100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:13<00:00,  3.12it/s]


In [9]:
# result1.to_excel('分红占比.xlsx')


In [9]:
# 计算加权后影响
result1['分红占股价'] = result1['分红'] / result1['上期收盘价']
result1['权重*分红占比'] = result1['权重'] * result1['分红占股价']
# print(result1.head())

In [10]:
#选出当月还没实施分红的
already = result1[result1['除权除息日']>=nowtime]
result1 = result1.drop(already.index)

In [11]:
# 汇总到每个交割日
result2 = result1.groupby(by='对应交割日')['简称', '权重*分红占比'].agg({'简称': np.count_nonzero, '权重*分红占比': np.sum})
result2.columns = ['分红股票数', '分红影响']
result2['分红影响%'] = result2['分红影响'] * 100

C:\Users\25286\AppData\Local\Temp/ipykernel_11220/1183847253.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result2 = result1.groupby(by='对应交割日')['简称', '权重*分红占比'].agg({'简称': np.count_nonzero, '权重*分红占比': np.sum})


In [12]:
# 获取价格日沪深300收盘价
hsclose = pd.read_excel('data/沪深300分红.xlsx', sheet_name='交割日', index_col='交割日期')
result3 = pd.merge(result2, hsclose,how='left', left_index=True, right_index=True)
result3.pop('序号')
result3['沪深300上期收盘价'] = result3['沪深300收盘价'].shift(1)
result3['预计基差影响'] = result3['分红影响'] * result3['沪深300上期收盘价']

In [13]:
result3.to_excel(f'分红对期货基差的影响估计{datetime.date.today()}.xlsx')

In [14]:
result3.reset_index(inplace=True)
result3['年月'] = result3['对应交割日'].map(lambda x: int(x.strftime('%Y%m')))
result3.set_index(keys=['年月'], drop=False, inplace=True)
# print(result3)

In [15]:
for x in range(2019,2023):
    for y in range(12):
        yearmonths = x*100 + y+1
        try:
            result3.loc[yearmonths,  '预计基差影响']
        except KeyError:
            result3.loc[yearmonths,  '预计基差影响'] = 0
result3.fillna(0, inplace=True)

In [16]:
## 设置当前月份
month = yearmonth.month
now = int(yearmonth.strftime('%Y%m'))
close = result3.loc[np.max(result3.index), '沪深300收盘价']
def biasestimate(mon):
    bias = 0
    for x in range(mon):
        if month+x+1 <= 12:
            bias += (result3.loc[now-200+x+1,  '预计基差影响'] + result3.loc[now-100+x+1,  '预计基差影响'])/2
        else:
            nmonth = month+x+1-12
            if (now-100-month+nmonth) in result3.index.values:
                a = result3.loc[now-100-month+nmonth,  '预计基差影响']
            else:
                a = 0
            if (now-month+nmonth) in result3.index.values:
                b = result3.loc[now-month+nmonth,  '预计基差影响']
            else:
                b = 0        
            bias += (a + b)/2
    return bias

if now <= np.max(result3.index):
    nowbias = ((result3.loc[now-200,  '预计基差影响'] +
            result3.loc[now-100,  '预计基差影响'])/2 - result3.loc[now,  '预计基差影响'])
else:
    nowbias = (result3.loc[now-200,  '预计基差影响'] + result3.loc[now-100,  '预计基差影响'])/2
    
alreadybias = np.sum(already['权重*分红占比']) * close
nowbias2 = max([alreadybias, nowbias])
nextbias = biasestimate(1) + nowbias2
nextq1 = biasestimate(3 - (month+1)%3+1) + nowbias2
nextq2 = biasestimate(6 - (month+1)%3+1) + nowbias2
biases = [nowbias2, nextbias, nextq1, nextq2]

In [17]:
# 期货名称
from dateutil.relativedelta import relativedelta as drr

first = 'IF'+ yearmonth.strftime('%y%m')
second = 'IF'+ (yearmonth + drr(months = 1)).strftime('%y%m')
third = 'IF' + (yearmonth + drr(months = (4 - (yearmonth.month+1)%3 ))).strftime('%y%m')
fourth = 'IF' + (yearmonth + drr(months= (7- (yearmonth.month+1)%3 ))).strftime('%y%m')

result4 = pd.DataFrame(biases, index=[first, second, third, fourth], columns=['预期基差'])
print(result4)

             预期基差
IF2207  29.585313
IF2208  42.197702
IF2209  47.656448
IF2212  52.439150


In [18]:
print(nowbias)
print(alreadybias)

29.58531342298943
0.0
